In [2]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%

## Data Preparation

In [24]:
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import RandomizedSearchCV, train_test_split;
import numpy;
import pandas;
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier;
import sys;
from keras.utils import to_categorical;
REMOTE_MODULE_PATH = '/home/macbookretina/automatic-music-genre-classification/feature_extraction_deep_learning'
# LOCAL_MODULE_PATH = '/Users/macbookretina/Desktop/automatic-music-genre-classification/feature_extraction_deep_learning'
sys.path.insert(1, REMOTE_MODULE_PATH)
from custom_module.utilities import *

In [25]:
features = pandas.read_csv(MOUNTED_DATASET_PATH + '/data/cleaned_features.csv', index_col=0)
features

,genre_label,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,0,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,0,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,0,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,0,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,0,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,0,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,2,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,2,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [26]:
y = features.loc[:,'genre_label']
X = features.loc[:, features.columns.difference(['genre_label'])]
X

,lpc_1,lpc_2,lpc_3,lpc_4,mean_beats,mean_beats_timestamp,mean_db4_cA4,mean_db4_cD1,mean_db4_cD2,mean_db4_cD3,...,var_spec_centroid,var_spec_contrast_1,var_spec_contrast_2,var_spec_contrast_3,var_spec_contrast_4,var_spec_contrast_5,var_spec_contrast_6,var_spec_contrast_7,var_spec_rolloff,var_zcr
0,1.0,-1.178271,0.647402,-0.352129,649.120690,15.072553,-0.070889,2.706283e-09,-1.245192e-04,-0.000069,...,4.185230e+05,17.861575,21.121083,25.873245,13.038216,6.962883,7.988678,14.555389,1.564158e+06,0.002891
1,1.0,-1.086489,0.542333,-0.396561,617.465116,14.337512,-0.000740,8.956381e-06,-9.212386e-05,0.000984,...,9.392262e+05,29.258739,24.407069,23.032866,18.185633,12.981837,12.461155,107.433159,2.076888e+06,0.005135
2,1.0,-1.365355,0.560194,-0.172092,631.428571,14.661743,-0.011143,-1.990486e-09,-7.116798e-06,0.000019,...,6.335591e+05,14.853086,20.595163,23.554291,21.775710,11.043003,11.478725,17.035474,3.405313e+06,0.002481
3,1.0,-1.240807,0.505005,-0.190461,619.227273,14.378429,-0.002008,3.508618e-05,-2.028877e-05,-0.001993,...,4.953543e+05,25.631542,17.320487,26.199990,13.617381,7.566584,8.344646,142.327995,1.811020e+06,0.002283
4,1.0,-0.962111,0.201135,-0.206167,631.382979,14.660684,-0.001342,3.828920e-05,-4.344582e-05,0.000076,...,1.073302e+06,35.138048,34.537389,32.614328,30.967176,19.603025,10.813280,102.154238,4.354818e+06,0.006124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3294,1.0,-1.878301,1.492194,-0.602686,1246.841270,14.475799,0.000255,-3.682523e-07,1.021450e-05,-0.000065,...,6.717626e+05,25.312044,16.848874,13.235958,17.875383,10.292653,7.975512,70.855880,3.494444e+06,0.001376
3295,1.0,-0.757695,0.322183,-0.531011,1269.770833,14.742011,0.000069,-3.853609e-07,3.565250e-05,-0.000009,...,9.118337e+06,36.373645,31.511738,26.118394,30.467805,48.009289,32.720426,98.557969,2.956056e+07,0.007543
3296,1.0,-1.862517,1.301521,-0.433298,1254.981818,14.570310,0.000784,-1.326012e-06,6.062675e-07,-0.000024,...,9.370093e+05,47.880472,15.887094,19.648933,20.805391,14.361217,9.615398,48.662283,6.126479e+06,0.000358
3297,1.0,-1.882439,1.470879,-0.573606,1272.877551,14.778080,-0.000206,-3.779051e-07,2.349206e-05,-0.000132,...,1.390328e+06,32.331749,18.345547,18.593669,21.698056,14.748304,8.958216,19.165002,5.990872e+06,0.001409


In [27]:
y.value_counts(normalize=True)

2    0.333434
0    0.333434
1    0.333131
Name: genre_label, dtype: float64

In [28]:
wavelet_predictors = X.filter(regex=(r'.+_db[458]{1}_.+'))
wavelet_predictor_labels = wavelet_predictors.columns.values

In [29]:
preprocess_sp = standardization_pipeline(
    wavelet_predictor_labels,
    [], wavelet_predictor_labels)
preprocess_np = normalization_pipeline(
    wavelet_predictor_labels)

### Split & Transform Data

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    wavelet_predictors, y, test_size=0.2, shuffle=True)

In [47]:
pipeline_estimator_sp = standardization_pipeline(
    wavelet_predictor_labels,
    [], wavelet_predictor_labels
).fit(X_train)
# X_train_sp = pipeline_estimator_sp.transform(X_train)
X_test_sp = pipeline_estimator_sp.transform(X_test)
X_test_sp.shape

(660, 108, 1)

In [48]:
pipeline_estimator_np = normalization_pipeline(
    wavelet_predictor_labels).fit(X_train)
# X_train_np = pipeline_estimator_np.transform(X_train)
X_test_np = pipeline_estimator_np.transform(X_test)
X_test_np.shape

(660, 108, 1)

In [49]:
# X_train_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
#                                   ).fit_transform(X_train)
# X_test_sp = standardization_pipeline(wavelet_predictor_labels, 
#                                    [], wavelet_predictor_labels
#                                   ).fit_transform(X_test)
# X_test_sp.shape

In [50]:
# X_train_np = normalization_pipeline(wavelet_predictor_labels
#                                   ).fit_transform(X_train)
# X_test_np = normalization_pipeline(wavelet_predictor_labels
#                                   ).fit_transform(X_test)
# X_test_np.shape

In [51]:
_, ncols = X_train.shape

In [52]:
# one hot encoding our labels
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [53]:
y_train.shape, y_test.shape

((2639, 3), (660, 3))

## Predictive Modelling

In [54]:
early_stopping_cb = callbacks.EarlyStopping(patience=10,
                                        restore_best_weights=True)

In [55]:
param_dist = {
    'clf__n_hidden': [0,1,2,3,4,5],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100)
}

### FNN With Standardized Input

In [56]:
# wrap the function with keras wrapper
fnn_1 = KerasClassifier(build_fn=set_shape_create_model('fnn', ncols))

In [57]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', fnn_1)
])

In [58]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [59]:
# %%timeit
# fit model
search_1 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 108)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                3488      
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 99        
Total params: 4,643
Trainable params: 4,643
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 6ms/step - loss: 1.1624 - accuracy: 0.4349 - val_accuracy: 0.4803 - val_loss: 1.1042
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0779 - accuracy: 0.4724 - val_accuracy

55/55 [==============================] - 0s 2ms/step - loss: 1.0606 - accuracy: 0.4821 - val_accuracy: 0.5106 - val_loss: 1.0599
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0251 - accuracy: 0.5003 - val_accuracy: 0.5197 - val_loss: 1.0345
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0018 - accuracy: 0.5105 - val_accuracy: 0.5318 - val_loss: 1.0167
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9845 - accuracy: 0.5168 - val_accuracy: 0.5424 - val_loss: 1.0032
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9710 - accuracy: 0.5264 - val_accuracy: 0.5500 - val_loss: 0.9922
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9595 - accuracy: 0.5276 - val_accuracy: 0.5485 - val_loss: 0.9831
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9498 - accuracy: 0.5355 - val_accuracy: 0.5515 - val_loss: 0.9754
Epoch 9/100
55/55 [======

Epoch 60/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8194 - accuracy: 0.6265 - val_accuracy: 0.5788 - val_loss: 0.8748
Epoch 61/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8184 - accuracy: 0.6271 - val_accuracy: 0.5788 - val_loss: 0.8741
Epoch 62/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8174 - accuracy: 0.6265 - val_accuracy: 0.5788 - val_loss: 0.8734
Epoch 63/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8164 - accuracy: 0.6242 - val_accuracy: 0.5773 - val_loss: 0.8728
Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8155 - accuracy: 0.6276 - val_accuracy: 0.5773 - val_loss: 0.8722
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8146 - accuracy: 0.6282 - val_accuracy: 0.5788 - val_loss: 0.8716
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8136 - accuracy: 0.6288 - val_accuracy: 0.5788 - val_loss: 0.8709
Epoch 

Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8138 - accuracy: 0.6290 - val_accuracy: 0.5955 - val_loss: 0.8651
Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8130 - accuracy: 0.6290 - val_accuracy: 0.5970 - val_loss: 0.8645
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8121 - accuracy: 0.6273 - val_accuracy: 0.5955 - val_loss: 0.8640
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8113 - accuracy: 0.6301 - val_accuracy: 0.5970 - val_loss: 0.8634
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8106 - accuracy: 0.6295 - val_accuracy: 0.5970 - val_loss: 0.8628
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8098 - accuracy: 0.6313 - val_accuracy: 0.5970 - val_loss: 0.8623
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8090 - accuracy: 0.6295 - val_accuracy: 0.5970 - val_loss: 0.8617
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.8799 - accuracy: 0.5793 - val_accuracy: 0.6212 - val_loss: 0.8562
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8251 - accuracy: 0.6197 - val_accuracy: 0.6439 - val_loss: 0.8233
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7874 - accuracy: 0.6293 - val_accuracy: 0.6379 - val_loss: 0.8074
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7547 - accuracy: 0.6487 - val_accuracy: 0.6455 - val_loss: 0.7879
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7296 - accuracy: 0.6697 - val_accuracy: 0.6606 - val_loss: 0.7755
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7094 - accuracy: 0.6799 - val_accuracy: 0.6667 - val_loss: 0.7622
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6969 - accuracy: 0.6919 - val_accuracy: 0.6530 - val_loss: 0.7835
Epoch 9/100
55/55 [======

...

Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7698 - accuracy: 0.6710 - val_accuracy: 0.6318 - val_loss: 0.8172
Epoch 81/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7689 - accuracy: 0.6705 - val_accuracy: 0.6288 - val_loss: 0.8168
Epoch 82/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7679 - accuracy: 0.6716 - val_accuracy: 0.6318 - val_loss: 0.8163
Epoch 83/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7670 - accuracy: 0.6722 - val_accuracy: 0.6333 - val_loss: 0.8158
Epoch 84/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7661 - accuracy: 0.6727 - val_accuracy: 0.6348 - val_loss: 0.8152
Epoch 85/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7652 - accuracy: 0.6710 - val_accuracy: 0.6364 - val_loss: 0.8149
Epoch 86/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7643 - accuracy: 0.6705 - val_accuracy: 0.6364 - val_loss: 0.8143
Epoch 

Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7689 - accuracy: 0.6617 - val_accuracy: 0.6364 - val_loss: 0.8569
Epoch 5/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7388 - accuracy: 0.6731 - val_accuracy: 0.6258 - val_loss: 0.8449
Epoch 6/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7121 - accuracy: 0.6868 - val_accuracy: 0.6394 - val_loss: 0.8457
Epoch 7/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6889 - accuracy: 0.7038 - val_accuracy: 0.6227 - val_loss: 0.8491
Epoch 8/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6708 - accuracy: 0.7101 - val_accuracy: 0.6318 - val_loss: 0.8270
Epoch 9/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6479 - accuracy: 0.7152 - val_accuracy: 0.6409 - val_loss: 0.8526
Epoch 10/100
55/55 [==============================] - 0s 2ms/step - loss: 0.6342 - accuracy: 0.7300 - val_accuracy: 0.6333 - val_loss: 0.8317
Epoch 11/100

Epoch 64/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7499 - accuracy: 0.6612 - val_accuracy: 0.6621 - val_loss: 0.7876
Epoch 65/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7491 - accuracy: 0.6634 - val_accuracy: 0.6667 - val_loss: 0.7869
Epoch 66/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7480 - accuracy: 0.6617 - val_accuracy: 0.6682 - val_loss: 0.7856
Epoch 67/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7469 - accuracy: 0.6623 - val_accuracy: 0.6697 - val_loss: 0.7857
Epoch 68/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7463 - accuracy: 0.6652 - val_accuracy: 0.6697 - val_loss: 0.7850
Epoch 69/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7452 - accuracy: 0.6646 - val_accuracy: 0.6712 - val_loss: 0.7846
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7443 - accuracy: 0.6646 - val_accuracy: 0.6697 - val_loss: 0.7838
Epoch 

Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7444 - accuracy: 0.6612 - val_accuracy: 0.6470 - val_loss: 0.8020
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7435 - accuracy: 0.6595 - val_accuracy: 0.6439 - val_loss: 0.8011
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7429 - accuracy: 0.6589 - val_accuracy: 0.6470 - val_loss: 0.8008
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7419 - accuracy: 0.6595 - val_accuracy: 0.6455 - val_loss: 0.8001
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7413 - accuracy: 0.6555 - val_accuracy: 0.6455 - val_loss: 0.7996
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7402 - accuracy: 0.6612 - val_accuracy: 0.6470 - val_loss: 0.7989
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7393 - accuracy: 0.6629 - val_accuracy: 0.6515 - val_loss: 0.7982
Epoch 

Epoch 74/100
55/55 [==============================] - 0s 3ms/step - loss: 0.7399 - accuracy: 0.6699 - val_accuracy: 0.6561 - val_loss: 0.7847
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7390 - accuracy: 0.6722 - val_accuracy: 0.6561 - val_loss: 0.7847
Epoch 76/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7385 - accuracy: 0.6716 - val_accuracy: 0.6545 - val_loss: 0.7840
Epoch 77/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7376 - accuracy: 0.6733 - val_accuracy: 0.6561 - val_loss: 0.7838
Epoch 78/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7369 - accuracy: 0.6761 - val_accuracy: 0.6561 - val_loss: 0.7833
Epoch 79/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7362 - accuracy: 0.6744 - val_accuracy: 0.6545 - val_loss: 0.7828
Epoch 80/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7356 - accuracy: 0.6744 - val_accuracy: 0.6530 - val_loss: 0.7820
Epoch 

In [60]:
search_1.best_score_

0.6593406690526704

In [81]:
search_1.best_params_

{'clf__activation': 'selu',
 'clf__kernel_initializer': 'uniform',
 'clf__n_hidden': 3,
 'clf__n_neurons': 85,
 'clf__optimizer': 'adagrad'}

In [82]:
search_1.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', ..., 'var_db8_cD6', 'var_db8_cD7'],
      dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_without_outliers', Pipelin..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8dc4143278>)])

### FNN With Normalized Input

In [61]:
# wrap the function with keras wrapper
fnn_2 = KerasClassifier(set_shape_create_model('fnn_np', ncols))

In [62]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', fnn_2)
])

In [63]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [64]:
# %%timeit
# fit model
search_2 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_33 (Flatten)         (None, 108)               0         
_________________________________________________________________
dense_127 (Dense)            (None, 3)                 327       
Total params: 327
Trainable params: 327
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
55/55 [==============================] - 0s 3ms/step - loss: 1.1030 - accuracy: 0.3576 - val_accuracy: 0.3788 - val_loss: 1.0925
Epoch 2/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0779 - accuracy: 0.4048 - val_accuracy: 0.4394 - val_loss: 1.0755
Epoch 3/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0647 - accuracy: 0.4639 - val_accuracy: 0.4561 - val_loss: 1.0602
Epoch 4/100
55/55 [==============================] - 0s 2ms/step - loss: 1.0547 - accuracy: 0.50

55/55 [==============================] - 0s 2ms/step - loss: 0.9869 - accuracy: 0.5560 - val_accuracy: 0.4955 - val_loss: 0.9972
Epoch 12/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9840 - accuracy: 0.5571 - val_accuracy: 0.5424 - val_loss: 0.9871
Epoch 13/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9752 - accuracy: 0.5651 - val_accuracy: 0.5136 - val_loss: 0.9897
Epoch 14/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9699 - accuracy: 0.5640 - val_accuracy: 0.4576 - val_loss: 1.0006
Epoch 15/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9695 - accuracy: 0.5674 - val_accuracy: 0.4833 - val_loss: 0.9872
Epoch 16/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9664 - accuracy: 0.5560 - val_accuracy: 0.5121 - val_loss: 0.9796
Epoch 17/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9602 - accuracy: 0.5668 - val_accuracy: 0.5364 - val_loss: 0.9713
Epoch 18/100
55/55 

Epoch 69/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8792 - accuracy: 0.6015 - val_accuracy: 0.5894 - val_loss: 0.8979
Epoch 70/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8799 - accuracy: 0.6043 - val_accuracy: 0.5955 - val_loss: 0.8945
Epoch 71/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8796 - accuracy: 0.6015 - val_accuracy: 0.5606 - val_loss: 0.9085
Epoch 72/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8776 - accuracy: 0.6055 - val_accuracy: 0.5439 - val_loss: 0.9116
Epoch 73/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8778 - accuracy: 0.6072 - val_accuracy: 0.5848 - val_loss: 0.8981
Epoch 74/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8756 - accuracy: 0.6020 - val_accuracy: 0.5712 - val_loss: 0.8984
Epoch 75/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8758 - accuracy: 0.6003 - val_accuracy: 0.5561 - val_loss: 0.9080
Epoch 

55/55 [==============================] - 0s 2ms/step - loss: 0.9910 - accuracy: 0.5560 - val_accuracy: 0.5182 - val_loss: 0.9992
Epoch 27/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9890 - accuracy: 0.5543 - val_accuracy: 0.5106 - val_loss: 1.0000
Epoch 28/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9874 - accuracy: 0.5509 - val_accuracy: 0.5242 - val_loss: 0.9951
Epoch 29/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9856 - accuracy: 0.5526 - val_accuracy: 0.5227 - val_loss: 0.9941
Epoch 30/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9839 - accuracy: 0.5583 - val_accuracy: 0.5258 - val_loss: 0.9915
Epoch 31/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9824 - accuracy: 0.5571 - val_accuracy: 0.5258 - val_loss: 0.9885
Epoch 32/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9807 - accuracy: 0.5537 - val_accuracy: 0.5227 - val_loss: 0.9896
Epoch 33/100
55/55 

Epoch 84/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9405 - accuracy: 0.5691 - val_accuracy: 0.5485 - val_loss: 0.9449
Epoch 85/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9398 - accuracy: 0.5748 - val_accuracy: 0.5500 - val_loss: 0.9430
Epoch 86/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9394 - accuracy: 0.5674 - val_accuracy: 0.5485 - val_loss: 0.9448
Epoch 87/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9390 - accuracy: 0.5674 - val_accuracy: 0.5515 - val_loss: 0.9442
Epoch 88/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9386 - accuracy: 0.5731 - val_accuracy: 0.5530 - val_loss: 0.9425
Epoch 89/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9381 - accuracy: 0.5702 - val_accuracy: 0.5530 - val_loss: 0.9428
Epoch 90/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9377 - accuracy: 0.5742 - val_accuracy: 0.5515 - val_loss: 0.9423
Epoch 

...

55/55 [==============================] - 0s 3ms/step - loss: 0.8117 - accuracy: 0.6214 - val_accuracy: 0.6227 - val_loss: 0.8252
Epoch 18/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8431 - accuracy: 0.6055 - val_accuracy: 0.6167 - val_loss: 0.8226
Epoch 19/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7933 - accuracy: 0.6333 - val_accuracy: 0.5818 - val_loss: 0.8811
Epoch 20/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8064 - accuracy: 0.6208 - val_accuracy: 0.6152 - val_loss: 0.8206
Epoch 21/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8175 - accuracy: 0.6094 - val_accuracy: 0.6000 - val_loss: 0.8715
Epoch 22/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7943 - accuracy: 0.6350 - val_accuracy: 0.6015 - val_loss: 0.8373
Epoch 23/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8062 - accuracy: 0.6271 - val_accuracy: 0.6045 - val_loss: 0.8713
Epoch 24/100
55/55 

55/55 [==============================] - 0s 2ms/step - loss: 0.9592 - accuracy: 0.5378 - val_accuracy: 0.5273 - val_loss: 0.9439
Epoch 32/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9569 - accuracy: 0.5372 - val_accuracy: 0.5379 - val_loss: 0.9414
Epoch 33/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9570 - accuracy: 0.5355 - val_accuracy: 0.5333 - val_loss: 0.9401
Epoch 34/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9552 - accuracy: 0.5378 - val_accuracy: 0.5394 - val_loss: 0.9389
Epoch 35/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9539 - accuracy: 0.5395 - val_accuracy: 0.5303 - val_loss: 0.9382
Epoch 36/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9532 - accuracy: 0.5378 - val_accuracy: 0.5364 - val_loss: 0.9367
Epoch 37/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9522 - accuracy: 0.5435 - val_accuracy: 0.5303 - val_loss: 0.9364
Epoch 38/100
55/55 

Epoch 89/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9249 - accuracy: 0.5463 - val_accuracy: 0.5515 - val_loss: 0.9099
Epoch 90/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9246 - accuracy: 0.5475 - val_accuracy: 0.5545 - val_loss: 0.9094
Epoch 91/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9239 - accuracy: 0.5475 - val_accuracy: 0.5545 - val_loss: 0.9090
Epoch 92/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9237 - accuracy: 0.5463 - val_accuracy: 0.5545 - val_loss: 0.9087
Epoch 93/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9232 - accuracy: 0.5509 - val_accuracy: 0.5545 - val_loss: 0.9083
Epoch 94/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9225 - accuracy: 0.5497 - val_accuracy: 0.5455 - val_loss: 0.9109
Epoch 95/100
55/55 [==============================] - 0s 2ms/step - loss: 0.9226 - accuracy: 0.5469 - val_accuracy: 0.5636 - val_loss: 0.9077
Epoch 

Epoch 95/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8925 - accuracy: 0.5679 - val_accuracy: 0.5576 - val_loss: 0.9076
Epoch 96/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8918 - accuracy: 0.5600 - val_accuracy: 0.5591 - val_loss: 0.9072
Epoch 97/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8914 - accuracy: 0.5623 - val_accuracy: 0.5576 - val_loss: 0.9069
Epoch 98/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8907 - accuracy: 0.5662 - val_accuracy: 0.5576 - val_loss: 0.9067
Epoch 99/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8908 - accuracy: 0.5657 - val_accuracy: 0.5561 - val_loss: 0.9065
Epoch 100/100
55/55 [==============================] - 0s 1ms/step - loss: 0.8892 - accuracy: 0.5634
Model: "fnn_np"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_59 (Flatten)         (None, 108) 

55/55 [==============================] - 0s 2ms/step - loss: 0.8029 - accuracy: 0.6322 - val_accuracy: 0.5455 - val_loss: 0.9703
Epoch 28/100
55/55 [==============================] - 0s 2ms/step - loss: 0.8131 - accuracy: 0.6248 - val_accuracy: 0.6242 - val_loss: 0.8217
Epoch 29/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7911 - accuracy: 0.6345 - val_accuracy: 0.6167 - val_loss: 0.8677
Epoch 30/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7903 - accuracy: 0.6327 - val_accuracy: 0.5727 - val_loss: 0.8892
Epoch 31/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7895 - accuracy: 0.6282 - val_accuracy: 0.4773 - val_loss: 1.0467
Epoch 32/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7861 - accuracy: 0.6293 - val_accuracy: 0.6030 - val_loss: 0.8511
Epoch 33/100
55/55 [==============================] - 0s 2ms/step - loss: 0.7919 - accuracy: 0.6339 - val_accuracy: 0.4106 - val_loss: 1.2661
Epoch 34/100
55/55 

In [65]:
search_2.best_score_

0.6085638372046155

In [79]:
search_2.best_params_

{'clf__activation': 'selu',
 'clf__kernel_initializer': 'glorot_uniform',
 'clf__n_hidden': 2,
 'clf__n_neurons': 76,
 'clf__optimizer': 'adam'}

In [80]:
search_2.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', ..., 'var_db8_cD6', 'var_db8_cD7'],
      dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8dc41f0eb8>)])

### CNN With Standardized Input

In [66]:
param_dist = {
    'clf__n_hidden': [1, 2, 3],
    'clf__activation': ['relu', 'elu', 'selu'],
    'clf__optimizer': ['rmsprop','adam','adagrad'],
    'clf__kernel_initializer':['glorot_uniform','normal','uniform'],
    'clf__n_neurons': numpy.arange(1,100),
    'clf__filters': [16, 32, 64],
#     'clf__kernel_size': [3,4,5],
    'clf__dropout':[.1, .15, .2, .25]
}

In [67]:
# wrap the function with keras wrapper
cnn_1 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn', ncols))

In [68]:
pipeline_sp = Pipeline([
    ('preprocess', preprocess_sp),
    ('clf', cnn_1)
])

In [69]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_sp, param_dist, cv=3, n_jobs=1)

In [70]:
# %%timeit
# fit model
search_3 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_sp, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 106, 32)           128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 53, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 11, 96)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 9, 128)            36992   

Epoch 36/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7015 - accuracy: 0.6879 - val_accuracy: 0.6333 - val_loss: 0.8207
Epoch 37/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6969 - accuracy: 0.6964 - val_accuracy: 0.6273 - val_loss: 0.8119
Epoch 38/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6914 - accuracy: 0.6987 - val_accuracy: 0.6197 - val_loss: 0.8118
Epoch 39/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6897 - accuracy: 0.7089 - val_accuracy: 0.6242 - val_loss: 0.8121
Epoch 40/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6801 - accuracy: 0.7061 - val_accuracy: 0.6318 - val_loss: 0.8185
Epoch 41/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6877 - accuracy: 0.6976 - val_accuracy: 0.6227 - val_loss: 0.8141
Epoch 42/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6800 - accuracy: 0.7084 - val_accuracy: 0.6303 - val_loss: 0.8156

Epoch 27/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7360 - accuracy: 0.6597 - val_accuracy: 0.6273 - val_loss: 0.8225
Epoch 28/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7361 - accuracy: 0.6653 - val_accuracy: 0.6227 - val_loss: 0.8137
Epoch 29/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7303 - accuracy: 0.6699 - val_accuracy: 0.6379 - val_loss: 0.8157
Epoch 30/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7245 - accuracy: 0.6756 - val_accuracy: 0.6303 - val_loss: 0.8160
Epoch 31/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7242 - accuracy: 0.6687 - val_accuracy: 0.6288 - val_loss: 0.8147
Epoch 32/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7192 - accuracy: 0.6659 - val_accuracy: 0.6303 - val_loss: 0.8123
Epoch 33/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7129 - accuracy: 0.6795 - val_accuracy: 0.6288 - val_loss: 0.8122

Epoch 1/100
55/55 [==============================] - 1s 14ms/step - loss: 0.9651 - accuracy: 0.5060 - val_accuracy: 0.5742 - val_loss: 0.9428
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8560 - accuracy: 0.6015 - val_accuracy: 0.5970 - val_loss: 0.8717
Epoch 3/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7884 - accuracy: 0.6390 - val_accuracy: 0.6273 - val_loss: 0.8364
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 0.7143 - accuracy: 0.6868 - val_accuracy: 0.6303 - val_loss: 0.8402
Epoch 5/100
55/55 [==============================] - 1s 11ms/step - loss: 0.6758 - accuracy: 0.7027 - val_accuracy: 0.6030 - val_loss: 0.8307
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 0.6012 - accuracy: 0.7459 - val_accuracy: 0.6515 - val_loss: 0.8621
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.5204 - accuracy: 0.7766 - val_accuracy: 0.6394 - val_loss: 0.9500
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0002 - accuracy: 0.5037 - val_accuracy: 0.5621 - val_loss: 0.9042
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8788 - accuracy: 0.5992 - val_accuracy: 0.5924 - val_loss: 0.8655
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8184 - accuracy: 0.6242 - val_accuracy: 0.6439 - val_loss: 0.8391
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7660 - accuracy: 0.6578 - val_accuracy: 0.6500 - val_loss: 0.8019
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7229 - accuracy: 0.6788 - val_accuracy: 0.6758 - val_loss: 0.7887
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6660 - accuracy: 0.6987 - val_accuracy: 0.6500 - val_loss: 0.8328
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6567 - accuracy: 0.7084 - val_accuracy: 0.6742 - val_loss: 0.7951
Epoch 8/100


...

Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0049 - accuracy: 0.5077 - val_accuracy: 0.5742 - val_loss: 0.9409
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9208 - accuracy: 0.5708 - val_accuracy: 0.6061 - val_loss: 0.9097
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8913 - accuracy: 0.6072 - val_accuracy: 0.6197 - val_loss: 0.8924
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8659 - accuracy: 0.6157 - val_accuracy: 0.6106 - val_loss: 0.8806
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8476 - accuracy: 0.6231 - val_accuracy: 0.5985 - val_loss: 0.8713
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8300 - accuracy: 0.6282 - val_accuracy: 0.6030 - val_loss: 0.8679
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8262 - accuracy: 0.6356 - val_accuracy: 0.6076 - val_loss: 0.8577
Epoch 8/100


Epoch 67/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6536 - accuracy: 0.7295 - val_accuracy: 0.6606 - val_loss: 0.7915
Epoch 68/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6535 - accuracy: 0.7261 - val_accuracy: 0.6561 - val_loss: 0.7847
Epoch 69/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6523 - accuracy: 0.7210 - val_accuracy: 0.6591 - val_loss: 0.7841
Epoch 70/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6494 - accuracy: 0.7301 - val_accuracy: 0.6470 - val_loss: 0.7811
Epoch 71/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6505 - accuracy: 0.7261 - val_accuracy: 0.6727 - val_loss: 0.7873
Epoch 72/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6426 - accuracy: 0.7324 - val_accuracy: 0.6652 - val_loss: 0.7853
Epoch 73/100
55/55 [==============================] - 0s 7ms/step - loss: 0.6421 - accuracy: 0.7261 - val_accuracy: 0.6545 - val_loss: 0.7860
Epoch 

Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0132 - accuracy: 0.4719 - val_accuracy: 0.5455 - val_loss: 0.9166
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8658 - accuracy: 0.5748 - val_accuracy: 0.5606 - val_loss: 0.8632
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7881 - accuracy: 0.6390 - val_accuracy: 0.5879 - val_loss: 0.8806
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7348 - accuracy: 0.6669 - val_accuracy: 0.6061 - val_loss: 0.8440
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6958 - accuracy: 0.6941 - val_accuracy: 0.6348 - val_loss: 0.8123
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.6151 - accuracy: 0.7396 - val_accuracy: 0.6061 - val_loss: 0.9538
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.5428 - accuracy: 0.7737 - val_accuracy: 0.6121 - val_loss: 0.9402
Epoch 8/100


Epoch 1/100
83/83 [==============================] - 1s 7ms/step - loss: 0.9859 - accuracy: 0.5225 - val_accuracy: 0.5727 - val_loss: 0.9015
Epoch 2/100
83/83 [==============================] - 0s 5ms/step - loss: 0.9089 - accuracy: 0.5722 - val_accuracy: 0.6061 - val_loss: 0.8701
Epoch 3/100
83/83 [==============================] - 0s 5ms/step - loss: 0.8813 - accuracy: 0.5904 - val_accuracy: 0.6045 - val_loss: 0.8563
Epoch 4/100
83/83 [==============================] - 0s 5ms/step - loss: 0.8621 - accuracy: 0.6078 - val_accuracy: 0.6212 - val_loss: 0.8448
Epoch 5/100
83/83 [==============================] - 0s 5ms/step - loss: 0.8403 - accuracy: 0.6207 - val_accuracy: 0.6045 - val_loss: 0.8428
Epoch 6/100
83/83 [==============================] - 0s 5ms/step - loss: 0.8364 - accuracy: 0.6237 - val_accuracy: 0.6227 - val_loss: 0.8330
Epoch 7/100
83/83 [==============================] - 0s 6ms/step - loss: 0.8314 - accuracy: 0.6165 - val_accuracy: 0.6167 - val_loss: 0.8273
Epoch 8/100
8

Epoch 59/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7174 - accuracy: 0.6897 - val_accuracy: 0.6500 - val_loss: 0.7740
Epoch 60/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7195 - accuracy: 0.6772 - val_accuracy: 0.6439 - val_loss: 0.7731
Epoch 61/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7179 - accuracy: 0.6874 - val_accuracy: 0.6439 - val_loss: 0.7733
Epoch 62/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7143 - accuracy: 0.6817 - val_accuracy: 0.6424 - val_loss: 0.7729
Epoch 63/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7158 - accuracy: 0.6847 - val_accuracy: 0.6439 - val_loss: 0.7728
Epoch 64/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7161 - accuracy: 0.6764 - val_accuracy: 0.6409 - val_loss: 0.7715
Epoch 65/100
83/83 [==============================] - 0s 5ms/step - loss: 0.7191 - accuracy: 0.6775 - val_accuracy: 0.6394 - val_loss: 0.7715
Epoch 

In [71]:
search_3.best_score_

0.6354679715321705

In [77]:
search_3.best_params_

{'clf__activation': 'selu',
 'clf__dropout': 0.2,
 'clf__filters': 16,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 58,
 'clf__optimizer': 'adagrad'}

In [78]:
search_3.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', ..., 'var_db8_cD6', 'var_db8_cD7'],
      dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_without_outliers', Pipelin..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8d9fd75588>)])

### CNN With Normalized Input

In [72]:
# wrap the function with keras wrapper
cnn_2 = KerasClassifier(build_fn=set_shape_create_cnn_model('cnn_np', ncols))

In [73]:
pipeline_np = Pipeline([
    ('preprocess', preprocess_np),
    ('clf', cnn_1)
])

In [74]:
# if you're not using a GPU, you can set n_jobs to something other than 1
rscv = RandomizedSearchCV(pipeline_np, param_dist, cv=3, n_jobs=1)

In [75]:
# %%timeit
# fit model
search_4 = rscv.fit(X_train, y_train, clf__batch_size=32,
                    clf__validation_data=(X_test_np, y_test),
                    clf__epochs=100, clf__callbacks=[early_stopping_cb])

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_89 (Conv1D)           (None, 106, 32)           128       
_________________________________________________________________
max_pooling1d_89 (MaxPooling (None, 53, 32)            0         
_________________________________________________________________
conv1d_90 (Conv1D)           (None, 51, 64)            6208      
_________________________________________________________________
max_pooling1d_90 (MaxPooling (None, 25, 64)            0         
_________________________________________________________________
conv1d_91 (Conv1D)           (None, 23, 96)            18528     
_________________________________________________________________
max_pooling1d_91 (MaxPooling (None, 11, 96)            0         
_________________________________________________________________
conv1d_92 (Conv1D)           (None, 9, 128)            36992   

Epoch 1/100
55/55 [==============================] - 1s 13ms/step - loss: 1.0993 - accuracy: 0.3489 - val_accuracy: 0.3197 - val_loss: 1.0974
Epoch 2/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0950 - accuracy: 0.3710 - val_accuracy: 0.4076 - val_loss: 1.0954
Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0928 - accuracy: 0.3875 - val_accuracy: 0.4106 - val_loss: 1.0929
Epoch 4/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0895 - accuracy: 0.4210 - val_accuracy: 0.4076 - val_loss: 1.0890
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0810 - accuracy: 0.4466 - val_accuracy: 0.4379 - val_loss: 1.0806
Epoch 6/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0708 - accuracy: 0.4648 - val_accuracy: 0.4561 - val_loss: 1.0684
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0514 - accuracy: 0.4926 - val_accuracy: 0.4894 - val_loss: 1.0444
Epoch 

Epoch 3/100
55/55 [==============================] - 1s 10ms/step - loss: 0.9380 - accuracy: 0.5321 - val_accuracy: 0.5333 - val_loss: 0.9359
Epoch 4/100
55/55 [==============================] - 1s 11ms/step - loss: 0.9007 - accuracy: 0.5645 - val_accuracy: 0.5530 - val_loss: 0.9248
Epoch 5/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8760 - accuracy: 0.5924 - val_accuracy: 0.4985 - val_loss: 1.0029
Epoch 6/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8671 - accuracy: 0.5924 - val_accuracy: 0.4182 - val_loss: 1.1251
Epoch 7/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8519 - accuracy: 0.6123 - val_accuracy: 0.5258 - val_loss: 0.9863
Epoch 8/100
55/55 [==============================] - 1s 10ms/step - loss: 0.8356 - accuracy: 0.6111 - val_accuracy: 0.5227 - val_loss: 0.9526
Epoch 9/100
55/55 [==============================] - 1s 11ms/step - loss: 0.8150 - accuracy: 0.6310 - val_accuracy: 0.5424 - val_loss: 0.9241
Epoch 

Epoch 14/100
55/55 [==============================] - 1s 11ms/step - loss: 0.7263 - accuracy: 0.6799 - val_accuracy: 0.4939 - val_loss: 1.0791
Epoch 15/100
55/55 [==============================] - 1s 12ms/step - loss: 0.7109 - accuracy: 0.6799 - val_accuracy: 0.5333 - val_loss: 1.1474
Epoch 16/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6940 - accuracy: 0.6998 - val_accuracy: 0.4621 - val_loss: 1.3925
Epoch 17/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6890 - accuracy: 0.6907 - val_accuracy: 0.5167 - val_loss: 1.0590
Epoch 18/100
55/55 [==============================] - 1s 12ms/step - loss: 0.6725 - accuracy: 0.6993 - val_accuracy: 0.5500 - val_loss: 1.0485
Epoch 19/100
55/55 [==============================] - 0s 5ms/step - loss: 0.8453 - accuracy: 0.5947
Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_109 (Conv1D)          (None, 106,

55/55 [==============================] - 1s 10ms/step - loss: 1.0936 - accuracy: 0.4133 - val_accuracy: 0.4955 - val_loss: 1.0597
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 1.0191 - accuracy: 0.5048 - val_accuracy: 0.4970 - val_loss: 0.9670
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9409 - accuracy: 0.5475 - val_accuracy: 0.5439 - val_loss: 0.9202
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9198 - accuracy: 0.5543 - val_accuracy: 0.5667 - val_loss: 0.9109
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8992 - accuracy: 0.5753 - val_accuracy: 0.5409 - val_loss: 0.9114
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8810 - accuracy: 0.5867 - val_accuracy: 0.5879 - val_loss: 0.8857
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8782 - accuracy: 0.5884 - val_accuracy: 0.6015 - val_loss: 0.8671
Epoch 8/100
55/55 [=====

...

Epoch 5/100
55/55 [==============================] - 0s 6ms/step - loss: 0.9989 - accuracy: 0.5392 - val_accuracy: 0.4758 - val_loss: 1.0236
Epoch 6/100
55/55 [==============================] - 0s 7ms/step - loss: 0.9819 - accuracy: 0.5602 - val_accuracy: 0.4864 - val_loss: 1.0156
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.9694 - accuracy: 0.5676 - val_accuracy: 0.5121 - val_loss: 1.0004
Epoch 8/100
55/55 [==============================] - 0s 6ms/step - loss: 0.9615 - accuracy: 0.5710 - val_accuracy: 0.5182 - val_loss: 0.9944
Epoch 9/100
55/55 [==============================] - 0s 6ms/step - loss: 0.9530 - accuracy: 0.5898 - val_accuracy: 0.4652 - val_loss: 1.0122
Epoch 10/100
55/55 [==============================] - 0s 6ms/step - loss: 0.9508 - accuracy: 0.5659 - val_accuracy: 0.4848 - val_loss: 1.0062
Epoch 11/100
55/55 [==============================] - 0s 6ms/step - loss: 0.9378 - accuracy: 0.5670 - val_accuracy: 0.5015 - val_loss: 0.9877
Epoch 12/10

Epoch 64/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8852 - accuracy: 0.5952 - val_accuracy: 0.5848 - val_loss: 0.8965
Epoch 65/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8802 - accuracy: 0.5998 - val_accuracy: 0.5742 - val_loss: 0.8945
Epoch 66/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8772 - accuracy: 0.5975 - val_accuracy: 0.5803 - val_loss: 0.8950
Epoch 67/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8826 - accuracy: 0.5918 - val_accuracy: 0.5848 - val_loss: 0.8954
Epoch 68/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8759 - accuracy: 0.5924 - val_accuracy: 0.5758 - val_loss: 0.8924
Epoch 69/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8785 - accuracy: 0.5947 - val_accuracy: 0.5864 - val_loss: 0.8941
Epoch 70/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8790 - accuracy: 0.5981 - val_accuracy: 0.5561 - val_loss: 0.8993
Epoch 

55/55 [==============================] - 1s 19ms/step - loss: 1.0933 - accuracy: 0.3943 - val_accuracy: 0.4348 - val_loss: 1.0837
Epoch 2/100
55/55 [==============================] - 0s 7ms/step - loss: 1.0751 - accuracy: 0.4665 - val_accuracy: 0.4061 - val_loss: 1.0732
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 1.0629 - accuracy: 0.4795 - val_accuracy: 0.4773 - val_loss: 1.0578
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 1.0476 - accuracy: 0.5102 - val_accuracy: 0.4985 - val_loss: 1.0438
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 1.0371 - accuracy: 0.5222 - val_accuracy: 0.5015 - val_loss: 1.0320
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 1.0217 - accuracy: 0.5114 - val_accuracy: 0.5167 - val_loss: 1.0205
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 1.0110 - accuracy: 0.5392 - val_accuracy: 0.5288 - val_loss: 1.0087
Epoch 8/100
55/55 [=====

Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0874 - accuracy: 0.3951 - val_accuracy: 0.3879 - val_loss: 1.0593
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9441 - accuracy: 0.5173 - val_accuracy: 0.4742 - val_loss: 1.0424
Epoch 3/100
55/55 [==============================] - 0s 7ms/step - loss: 0.8934 - accuracy: 0.5725 - val_accuracy: 0.5030 - val_loss: 0.9811
Epoch 4/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8817 - accuracy: 0.5674 - val_accuracy: 0.4606 - val_loss: 1.1035
Epoch 5/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8204 - accuracy: 0.6106 - val_accuracy: 0.5258 - val_loss: 1.0073
Epoch 6/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7939 - accuracy: 0.6271 - val_accuracy: 0.4788 - val_loss: 1.1779
Epoch 7/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7819 - accuracy: 0.6259 - val_accuracy: 0.3970 - val_loss: 1.3062
Epoch 8/100


Epoch 1/100
55/55 [==============================] - 1s 10ms/step - loss: 1.0807 - accuracy: 0.3739 - val_accuracy: 0.4788 - val_loss: 1.0173
Epoch 2/100
55/55 [==============================] - 0s 8ms/step - loss: 0.9079 - accuracy: 0.5693 - val_accuracy: 0.5030 - val_loss: 0.9403
Epoch 3/100
55/55 [==============================] - 0s 8ms/step - loss: 0.8956 - accuracy: 0.5653 - val_accuracy: 0.5258 - val_loss: 0.9169
Epoch 4/100
55/55 [==============================] - 0s 7ms/step - loss: 0.8466 - accuracy: 0.6040 - val_accuracy: 0.5682 - val_loss: 0.9203
Epoch 5/100
55/55 [==============================] - 0s 7ms/step - loss: 0.8299 - accuracy: 0.6307 - val_accuracy: 0.5364 - val_loss: 0.9092
Epoch 6/100
55/55 [==============================] - 0s 8ms/step - loss: 0.7955 - accuracy: 0.6369 - val_accuracy: 0.5530 - val_loss: 0.9550
Epoch 7/100
55/55 [==============================] - 0s 7ms/step - loss: 0.7833 - accuracy: 0.6477 - val_accuracy: 0.5682 - val_loss: 0.8902
Epoch 8/100


In [76]:
search_4.best_score_

0.6331943948416151

In [83]:
search_4.best_params_

{'clf__activation': 'relu',
 'clf__dropout': 0.15,
 'clf__filters': 64,
 'clf__kernel_initializer': 'normal',
 'clf__n_hidden': 1,
 'clf__n_neurons': 79,
 'clf__optimizer': 'adam'}

In [84]:
search_4.best_estimator_

Pipeline(memory=None,
     steps=[('preprocess', Pipeline(memory=None,
     steps=[('addcolumnnames', AddColumnNames(columns=array(['mean_db4_cA4', 'mean_db4_cD1', ..., 'var_db8_cD6', 'var_db8_cD7'],
      dtype=object))), ('featureunion', FeatureUnion(n_jobs=1,
       transformer_list=[('predictors_all', Pipeline(memory=None..., ('clf', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8d9fc01d30>)])